In [1]:
from transformers import BartForQuestionAnswering, BartConfig

/home/phamngocthi/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
config = BartConfig()
config.d_model = 10
config.decoder_layers = 1
config.encoder_layers = 1
config.encoder_ffn_dim = 10
config.decoder_ffn_dim = 10
config.encoder_attention_heads = 2
config.decoder_attention_heads = 2

In [5]:
model = BartForQuestionAnswering(config)
model

BartForQuestionAnswering(
  (model): BartModel(
    (shared): Embedding(50265, 10, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 10, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 10)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=10, out_features=10, bias=True)
            (v_proj): Linear(in_features=10, out_features=10, bias=True)
            (q_proj): Linear(in_features=10, out_features=10, bias=True)
            (out_proj): Linear(in_features=10, out_features=10, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=10, out_features=10, bias=True)
          (fc2): Linear(in_features=10, out_features=10, bias=True)
          (final_layer_norm): LayerNorm((10,), eps=1e-05, elementwise_affine

In [6]:
import torch

In [18]:
# generate input model with size (batch, seq_len) -> (2, 10)
input_ids = torch.randint(0, 10, (2, 10))
attention_mask = torch.ones(2, 10)
decoder_input_ids = torch.randint(0, 10, (2, 10))
decoder_attention_mask = torch.ones(2, 10)

# start_positions (`torch.LongTensor` of shape `(batch_size,)`, *optional*):

# generate start_positions with size (batch) -> (2), end_positions with size (batch) -> (2)
start_positions = torch.randint(0, 10, (2,))
end_positions = start_positions + 1

start_positions, end_positions

(tensor([2, 3]), tensor([3, 4]))

In [19]:
output = model(
    input_ids=input_ids,
    attention_mask=attention_mask,
    decoder_input_ids=decoder_input_ids,
    decoder_attention_mask=decoder_attention_mask,
    start_positions=start_positions,
    end_positions=end_positions,
)

In [20]:
output.loss, output.start_logits, output.end_logits

(tensor(2.3740, grad_fn=<DivBackward0>),
 tensor([[-0.0234,  0.1112, -0.0530, -0.0052,  0.0133,  0.0273, -0.0557,  0.0742,
          -0.0291, -0.0514],
         [-0.0612, -0.0041, -0.0589, -0.0120,  0.0710, -0.0621, -0.0388, -0.0239,
          -0.0287, -0.1010]], grad_fn=<CloneBackward0>),
 tensor([[ 0.0056, -0.0922, -0.0151, -0.1345, -0.0516,  0.0772,  0.0885,  0.0338,
           0.0738,  0.0897],
         [-0.0286, -0.1100, -0.0257, -0.0993, -0.1241,  0.0649, -0.0610,  0.0710,
           0.0601,  0.0301]], grad_fn=<CloneBackward0>))

In [21]:
pred = output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)
pred

(tensor([1, 4]), tensor([9, 7]))

In [34]:
# test

dim = 20
num_labels = 2
a = torch.randint(0, 10, (2, 10, dim), dtype=torch.float32)

linear1 = torch.nn.Linear(dim, num_labels)

b = linear1(a)
b.shape

torch.Size([2, 10, 2])